In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Load All Pram Sold Data

In [ ]:
df = pd.read_json('../../data/kapiti_sold.json')

In [ ]:
df.info()

## Replace url with the real url

In [ ]:
df['url'] = df['url'].apply(lambda x: f"https://homes.co.nz/address{x}")

In [ ]:
df['date_month'] = pd.to_datetime(df['date']).dt.to_period('M')

## Show the data with duplicated address and only keep the highest price one

In [ ]:
df.loc[df.duplicated(subset=['address'], keep=False), ['address','date_month', 'price', 'url', 'decade_built']]

In [ ]:
df.sort_values(by=['price'], ascending=False, inplace=True)
df.drop_duplicates(subset=['address'], keep='first', inplace=True)

## Show Missing Data

In [ ]:
#msno.bar(df)

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
# Fill Missing Price Data with its mean value
#df['price'] = df['price'].replace(0, np.nan)
#df['price'] = df['price'].fillna(df['price'].mean())

## Display the data with missing price

In [ ]:
df.loc[df['price'].isna(), ['address', 'capital_value', 'land_value', 'improvement_value', 'display_estimated_lower_value','display_estimated_lower_value', 'url']]

## Drop price if missing

In [ ]:
df['price'] = df['price'].replace(0, np.nan)
df.dropna(subset = ['price'], inplace=True)

## calculate the average price group by suburb

In [ ]:
df.groupby(['suburb'])['price'].mean().sort_values(ascending=False).plot(kind='bar')


## Sales by month

In [ ]:
df.groupby(['date_month'])['date_month'].count().plot(kind='bar')

## Correlation of price

In [ ]:
columns_of_interest = ['price',
                       'sales_count',
                       'capital_value',
                       'land_value',
                       'improvement_value',
                       'num_car_spaces',
                       'bath_estimate',
                       'bed_estimate',
                       'floor_area',
                       'land_area',
                       'decade_built'
                       ]
correlation_matrix = df[columns_of_interest].corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()


## calculate percentage of (price - capital_value) / capital_value

In [ ]:
df['exceed_cv_percentage'] = (df['price'] - df['capital_value']) * 100 / df['capital_value']

## calculate the average exceed CV percentage group by suburb
df.groupby(['suburb'])['exceed_cv_percentage'].mean().sort_values(ascending=False).plot(kind='bar')


In [ ]:
# Group data by suburb and calculate relevant statistics
grouped_data = df.groupby('suburb').agg({
    'property_id': 'count',
    'price': ['mean', 'min', 'max'],
    'sales_count': 'sum',
    'floor_area': 'mean',
    'land_area': 'mean'
}).reset_index()

# Rename the columns for better readability
grouped_data.columns = ['Suburb', 'Number of Sales', 'Average Price', 'Min Price', 'Max Price',
                        'Total Sales Count', 'Average Floor Area', 'Average Land Area']


# Calculate price per square meter
grouped_data['Price per sqm'] = grouped_data['Average Price'] / grouped_data['Average Floor Area']

# Sort by price per square meter
grouped_data_sorted = grouped_data.sort_values(by='Price per sqm')

# Calculate the average decade built for each suburb
average_decade_built = df.groupby('suburb')['decade_built'].mean().reset_index()

# Merge the average decade built with the grouped data
merged_data = pd.merge(grouped_data_sorted, average_decade_built, left_on='Suburb', right_on='suburb')

print(merged_data)


In [ ]:
# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Resample the data by month and calculate the total sales amount and sales count for each month
monthly_sales_stats = df.resample('M', on='date').agg({
    'price': ['sum', 'count']
})

# Plot the total sales amount and sales count over time
fig, ax1 = plt.subplots(figsize=(12, 8))

# Make the first plot: total sales amount
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Sales Amount', color=color)
ax1.plot(monthly_sales_stats.index, monthly_sales_stats[('price', 'sum')], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a second y-axis for the sales count
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Sales Count', color=color)
ax2.plot(monthly_sales_stats.index, monthly_sales_stats[('price', 'count')], color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Set the title and show the plot
plt.title('Monthly Sales Amount and Count over Time')
plt.show()


In [ ]:
# Resample the data by month and street, then count the number of sales for each combination
monthly_sales_by_street = df.groupby(['suburb']).resample('M', on='date').size().unstack(0)

# Plot the sales count for each street over time
plt.figure(figsize=(15, 10))
for street in monthly_sales_by_street.columns:
    plt.plot(monthly_sales_by_street.index, monthly_sales_by_street[street], label=street)

plt.title('Monthly Sales Count by Street over Time')
plt.xlabel('Date')
plt.ylabel('Sales Count')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()